1. Configuración de Mario

In [1]:
 # Instalación de Mario y emulador
!pip install gym_super_mario_bros==7.3.0 nes_py

  Using cached gym_super_mario_bros-7.3.0-py2.py3-none-any.whl (198 kB)
  Using cached nes_py-8.1.8.tar.gz (76 kB)
  Using cached gym-0.21.0-py3-none-any.whl
  Using cached pyglet-1.5.11-py3-none-any.whl (1.1 MB)
  Created wheel for nes-py: filename=nes_py-8.1.8-cp39-cp39-win_amd64.whl size=47377 sha256=ab763687a574fa30f2472a955b23f95bd08aef0a3f7d094c96892663a911fa01
  Stored in directory: c:\users\coqu1t0\appdata\local\pip\cache\wheels\5b\37\c1\b0eb34358d6e26fac73cbedd57b76d450e3150ab8b2ae16e5e
Successfully built nes-py


In [1]:
# Importación del juego
import gym_super_mario_bros
#Importación del Joypad
from nes_py.wrappers import JoypadSpace
#Importación de controles simplificados
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [2]:
import gym
import numpy as np
from IPython import display
# Importación de Matplotlib para mostrar el impacto de los frames
import matplotlib.pyplot as plt
%matplotlib inline
# Configuración del juego
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [4]:
done = True
for step in range(5000):
    if done:
        state = env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    env.render()

env.close()

C:\ProgramData\Anaconda3\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] No se puede cambiar el modo de subproceso después de establecerlo
  warnings.warn(str(err))
C:\ProgramData\Anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


In [ ]:
# Crear una bandera - Para reiniciar o no
done = True
img = plt.imshow(env.render(mode='rgb_array'))
# Bucle a través de cada fotograma en el juego
for step in range(100000):
  # Empezar el juego para comenzar con: 
  if done:
    #Empezar juego
    env.reset()
  # Tomar acciones aleatorias
  action = env.action_space.sample()
  state, reward, done, info = env.step(action)
  # Mostrar el juego
  img.set_data(env.render(mode='rgb_array'))
  display.display(plt.gcf())
  display.clear_output(wait=True)
  #env.render('rgb_array')
# Cerrar juego
env.close()

2. Procesamiento del entorno

In [1]:
# Instalación pytorch
!pip3 install torch torchvision torchaudio

In [17]:
# Instalación librerias de Ml
!pip install stable-baselines3[extra]

In [22]:
pip install stable-baselines3

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Importación de GraySacaling Wrapper
from gym.wrappers import GrayScaleObservation
# Importación Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

In [4]:
# 1. Creación de la base de entornos
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# 2. Simplificando los controles
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# 3. Grayscale
env = GrayScaleObservation(env, keep_dim=True)
# 4. Envolver dentro del entorno ficticio
env = DummyVecEnv([lambda: env])
# 5. Apilar los frames
env = VecFrameStack(env, 4, channels_order='last')

In [5]:
state = env.reset()
state.shape
state, reward, done, info = env.step([env.action_space.sample()])

In [ ]:
# Usando Matplotlib para mostrar los frame del juego
plt.figure(figsize=(12,8))
for idx in range(state.shape[3]):
  plt.subplot(1,4,idx+1)
  plt.imshow(state[0][:,:,idx])
plt.show()

3. Entrenamiento

In [5]:
# Importar OS para la gestión de la ruta de archivo
import os
# Importar PPO para ALgos
from stable_baselines3 import PPO
# Importar Base Callback para guardar los modelos
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
# Función para guardar el modelo
# class TrainAndLoggingCallback(BaseCallback):

#     def __init__(self, check_freq, save_path, verbose=1):
#         super(TrainAndLoggingCallback, self).__init__(verbose)
#         self.check_freq = check_freq
#         self.save_path = save_path

#     def _init_callback(self):
#         if self.save_path is not None:
#             os.makedirs(self.save_path, exist_ok=True)

#     def _on_step(self):
#         if self.n_calls % self.check_freq == 0:
#             model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
#             self.model.save(model_path)

#         return True 

In [6]:
# Configuración de directorios
# CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [7]:
# Configuración del guardado del modelo
#callback = TrainAndLogginCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [8]:
# Inicialización del modelo
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [9]:
# Entrena el modelo AI, aquí es donde comienza el modelo AI a aprender
model.learn(total_timesteps=4096)

Logging to ./logs/PPO_1


C:\ProgramData\Anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


----------------------------
| time/              |     |
|    fps             | 124 |
|    iterations      | 1   |
|    time_elapsed    | 4   |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 24            |
|    iterations           | 2             |
|    time_elapsed         | 42            |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 1.5927944e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.00312      |
|    learning_rate        | 1e-06         |
|    loss                 | 130           |
|    n_updates            | 10            |
|    policy_gradient_loss | 1.49e-05      |
|    value_loss           | 333           |
-------------------------------------------
-----

In [10]:
# Guardar modelo
model.save("ppo_cartpole")

4. Prueba

In [6]:
# Cargar el modelo
model = PPO.load("ppo_cartpole")

In [ ]:
# Comienza el juego
state = env.reset()
# Bucle del juego
while True:
  env.render()
  action, _state = model.predict(state)
  state, reward, done, info = env.step(action)

C:\ProgramData\Anaconda3\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] No se puede cambiar el modo de subproceso después de establecerlo
  warnings.warn(str(err))
C:\ProgramData\Anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


In [ ]:
done = True
for step in range(5000):
    if done:
        state = env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    env.render()

env.close()